In [ ]:
# ================================
# Main: GestionSolped
# Autor: Paula Sierra, Henry Navarro - NetApplications
# Descripcion: Main principal del Bot
# Ultima modificacion: 24/11/2025
# Propiedad de Colsubsidio
# Cambios: Ajuste inicial para cumplimiento de estándar


# from NetApplications.PY.AutomatizacionGestionSolped.HU.HU03_ValidacionME53N import buscar_SolpedME53N
from Funciones.EscribirLog import WriteLog
import traceback
from Config.settings import RUTAS,SAP_CONFIG
session = conectar_sap(
        SAP_CONFIG["sistema"], 
        SAP_CONFIG["mandante"],
        SAP_CONFIG["user"],
        SAP_CONFIG["password"],
        SAP_CONFIG["idioma"]
        )

def Main_Pruebas3():
    try:
  
        session = conectar_sap(
         SAP_CONFIG["sistema"], 
         SAP_CONFIG["mandante"],
         SAP_CONFIG["user"],
         SAP_CONFIG["password"],
         SAP_CONFIG["idioma"]
         )
        GenerarOCDesdeSolped(session, "1300139102",2)  # Reemplaza con la Solped real:  1300139102  1300139269
            
    except Exception as e:
        error_text = traceback.format_exc()
        WriteLog(
            mensaje=f"ERROR GLOBAL: {e} | {error_text}",
            estado="ERROR",
            task_name="Main_GestionSOLPED",
            path_log=RUTAS["PathLogError"],
        )
        raise




 SAP Logon 750 ya se encuentra abierto
Iniciando conexion con SAP...
Abriendo nueva conexion a ERP-CORPORATIVO-CALIDAD...
Sesion existente reutilizada.
 Conectado correctamente a SAP.


In [2]:
# Cambios: Ajuste inicial para cumplimiento de estándar
# ================================
from HU.HU00_DespliegueAmbiente import EjecutarHU00
from HU.HU1_LoginSAP import ObtenerSesionActiva,conectar_sap,abrir_sap_logon
from HU.HU2_DescargaME5A import EjecutarHU02
from HU.HU03_ValidacionME53N import EjecutarHU03
from HU.HU5_GeneracionOC import GenerarOCDesdeSolped
from HU.HU4_DescargaOCME9F import descarga_OCME9F

# from NetApplications.PY.AutomatizacionGestionSolped.HU.HU03_ValidacionME53N import buscar_SolpedME53N
from Funciones.EscribirLog import WriteLog
import traceback
from Config.settings import RUTAS,SAP_CONFIG

import time
def Main_Pruebas3():
    try:

        session = conectar_sap(
        SAP_CONFIG["sistema"],
        SAP_CONFIG["mandante"],
        SAP_CONFIG["user"],
        SAP_CONFIG["password"],
        SAP_CONFIG["idioma"]
        )
        time.sleep(10)
        GenerarOCDesdeSolped(session, "1300138077",10)  # Reemplaza con la Solped real:  1300139102  1300139269
        return session   
    except Exception as e:
        error_text = traceback.format_exc()
        WriteLog(
            mensaje=f"ERROR GLOBAL: {e} | {error_text}",
            estado="ERROR",
            task_name="Main_GestionSOLPED",
            path_log=RUTAS["PathLogError"],
        )
        raise
session=Main_Pruebas3()


 SAP Logon 750 ya se encuentra abierto
Iniciando conexion con SAP...
Abriendo nueva conexion a ERP-CORPORATIVO-CALIDAD...
Sesion existente reutilizada.
 Conectado correctamente a SAP.
Transacción ME21N abierta con éxito.


In [3]:
def boton_existe(session,id):
    try:
        session.findById(id)
        return True
    except:
        return False

def EncontrarDynpros(session):
    import re

    wnd = session.ActiveWindow
    dynpros = set()
    cola = [wnd]
    patron = re.compile(r":[A-Z0-9_]+:(\d{4})")

    while cola:
        nodo = cola.pop(0)

        try:
            nid = nodo.Id
            encontrados = patron.findall(nid)
            for d in encontrados:
                dynpros.add(d)
        except:
            pass

        try:
            count = nodo.Children.Count
        except:
            continue

        for i in range(count):
            try:
                child = nodo.Children(i)
                cola.append(child)
            except:
                pass
    lista_dynpros = sorted(dynpros)

    if lista_dynpros:
        primer_valor = lista_dynpros[0]
        return primer_valor
    else:
        return("No se encontraron dynpros.")
ID=EncontrarDynpros(session)

In [6]:
IDENTI= f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ID}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14"    


In [8]:
import pyautogui

a=1
ventana=EncontrarDynpros(session)
ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
Posiciones=session.FindById(ID).Entries.Count
for i in range(Posiciones-2):
    print(i)
    combo=session.FindById(ID)
    item=session.FindById(ID).Entries(i)
    print(combo.Entries(i).value)
    combo.key=item.key
    # lista de posiciones de la solped
    #print(f"Navegando por la posición {i+1} de la Solped {solped}Esperando foco en pestaña textos..")
    # foco Select en la pestaña textos despues de imputacion 
    ventana=EncontrarDynpros(session)     
    ID= f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14"    
    print(f"ID Dinamico es: {ID}")
    session.FindById(ID).select()
    session.FindById(ID).setFocus()
    # ir a texto posicion con un TAB 
    pyautogui.hotkey("TAB")
    time.sleep(0.5)
    pyautogui.press('down') 
    for i in range(5):   
      # ir a pedido info con un flecha abajo
      pyautogui.press('down') 
      # entrar a editar texto con ctrl + enter
      pyautogui.hotkey("ctrl","enter")
      time.sleep(1)
      # Validar si el boton borrar texto existe
      Boton = boton_existe(session,"wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                      ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
      if Boton:
        print("Botón Borrar presente")
        # presionar Boton borrar texto
        botonBorrar=session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                      ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
        botonBorrar.press()
        # Punto en el texto
        puntoentexto = session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:"
                                      "SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/cntlTEXT_EDITOR_0201/shellcont/shell")
        puntoentexto.text = "."
        # paso al siguiiente item de la solped
        #ventana=EncontrarDynpros(session)     
        #ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
        #print(f"ID Dinamico es: {ID}")
        #session.FindById(ID).press()
        #session.FindById(ID).setFocus()
        time.sleep(0.5)
        for i in range(2):
          pyautogui.hotkey("TAB")
          time.sleep(0.5)
        pyautogui.hotkey("enter")
      
      else:
        print("Botón Borrar NO está en esta vista")



0
[ 10 ] 80041940 , SERVICIO BICI-TALLER X BICICLET COL BICI
ID Dinamico es: wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
1


AttributeError: FindById.Entries

In [41]:
import pyautogui


ventana=EncontrarDynpros(session)
ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
print(ID)
Posiciones=session.FindById(ID).Entries.Count
for i in range(Posiciones-2):
    print(i)
    ID2=ID
    ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
    combo=session.FindById(ID)
    item=session.FindById(ID).Entries(i)
    print(combo.Entries(i).value)
    combo.key=item.key
    if i>0:    
      session.FindById(ID2).selectedNode = "F01"
    # lista de posiciones de la solped
    #print(f"Navegando por la posición {i+1} de la Solped {solped}Esperando foco en pestaña textos..")
    # foco Select en la pestaña textos despues de imputacion 
    ventana=EncontrarDynpros(session)     
    ID= f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14"    
    print(f"ID Dinamico es: {ID}")
    session.FindById(ID).select()
    if i==0:
      ventana=EncontrarDynpros(session)     
      ID= f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14"         
    session.FindById(ID).setFocus()
    # ir a texto posicion con un TAB 
    pyautogui.hotkey("TAB")
    time.sleep(0.5)
    pyautogui.press('down') 
    for i in range(5):   
      # ir a pedido info con un flecha abajo
      pyautogui.press('down') 
      # entrar a editar texto con ctrl + enter
      pyautogui.hotkey("ctrl","enter")
      time.sleep(1)
      # Validar si el boton borrar texto existe
      Boton = boton_existe(session,"wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                      ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
      if Boton:
        print("Botón Borrar presente")
        # presionar Boton borrar texto
        botonBorrar=session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                      ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
        botonBorrar.press()
        # Punto en el texto
        puntoentexto = session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:"
                                      "SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/cntlTEXT_EDITOR_0201/shellcont/shell")
        puntoentexto.text = "."
        # paso al siguiiente item de la solped
        #ventana=EncontrarDynpros(session)     
        #ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
        #print(f"ID Dinamico es: {ID}")
        #session.FindById(ID).press()
        #session.FindById(ID).setFocus()
        time.sleep(0.5)
        for i in range(2):
          pyautogui.hotkey("TAB")
          time.sleep(0.5)
        pyautogui.hotkey("enter")
      
      else:
        print("Botón Borrar NO está en esta vista")



wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST
0
[ 10 ] 80041940 , SERVICIO BICI-TALLER X BICICLET COL BICI
ID Dinamico es: wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
1
[ 20 ] 80001039 , ALQUILER DE CARPAS


AttributeError: Property 'FindById.selectedNode' can not be set.

In [21]:
ID="wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/cntlTEXT_TYPES_0200/shell"
combo = session.FindById(ID)
for attr in dir(combo):
    print(attr)

ChangeCheckbox
ClickLink
CollapseNode
DefaultContextMenu
DoubleClickItem
DoubleClickNode
DumpState
EnsureVisibleHorizontalItem
ExpandNode
FindAllByName
FindAllByNameEx
FindById
FindByName
FindByNameEx
FindNodeKeyByPath
GetAbapImage
GetAllNodeKeys
GetCheckBoxState
GetColumnCol
GetColumnHeaders
GetColumnIndexFromName
GetColumnNames
GetColumnTitleFromName
GetColumnTitles
GetFocusedNodeKey
GetHierarchyLevel
GetHierarchyTitle
GetIsDisabled
GetIsEditable
GetIsHighLighted
GetItemHeight
GetItemLeft
GetItemStyle
GetItemText
GetItemTextColor
GetItemToolTip
GetItemTop
GetItemType
GetItemWidth
GetListTreeNodeItemCount
GetNextNodeKey
GetNodeAbapImage
GetNodeChildrenCount
GetNodeChildrenCountByPath
GetNodeHeight
GetNodeIndex
GetNodeItemHeaders
GetNodeKeyByPath
GetNodeLeft
GetNodePathByKey
GetNodeStyle
GetNodeTextByKey
GetNodeTextByPath
GetNodeTextColor
GetNodeToolTip
GetNodeTop
GetNodeWidth
GetNodesCol
GetParent
GetPreviousNodeKey
GetSelectedNodes
GetSelectionMode
GetStyleDescription
GetSubNodesCol


In [39]:
session.FindById(ID).selectedNode = "F01"



In [ ]:
combo=session.FindById(ID)
item=session.FindById(ID).Entries(5)
combo.key=item.key

In [ ]:
import win32com.client
import subprocess
import time
import os
from Config.settings import RUTAS
from HU.HU1_LoginSAP import ObtenerSesionActiva
from Funciones.ValidacionM21N import find_dyn6000_combo,boton_existe,find_button_by_text
import pyautogui

session.findById('wnd[0]/shellcont/shell/shellcont[1]/shell[1]').setFocus()
time.sleep(2)
for i in range(6):   
    pyautogui.hotkey("ctrl","TAB")
    time.sleep(0.5)
pyautogui.press('right')
pyautogui.press('right')
pyautogui.press('enter')
for i in range(6):   
    pyautogui.hotkey("ctrl","TAB")
    time.sleep(0.5)

In [ ]:
import pyautogui
import time
def boton_existe(session,id):
    try:
        session.findById(id)
        return True
    except:
        return False
for i in range(3): 
    print(i)  
    pyautogui.press('down') 
    pyautogui.hotkey("ctrl","enter")
    time.sleep(0.5)
    Boton = boton_existe(session,"wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                    ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
            
    if Boton:
        print("Botón Borrar presente")
        # Boton borrar texto
        botonBorrar=session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                    ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
        botonBorrar.press()
        # Punto en el texto
        puntoentexto = session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:"
                                    "SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/cntlTEXT_EDITOR_0201/shellcont/shell")
        puntoentexto.text = "."
        pyautogui.hotkey("shift","TAB")
        time.sleep(0.5)
        pyautogui.press('down') 
        #session.sendVKey(2)
    else:
        print("Botón Borrar NO está en esta vista")

0
Botón Borrar presente
1
Botón Borrar NO está en esta vista
2
Botón Borrar NO está en esta vista


In [ ]:
for i in range(4):   
    pyautogui.hotkey("TAB")
    time.sleep(0.5)

def boton_existe(session,id):
    try:
        session.findById(id)
        return True
    except:
        return False
      
for i in range(3):   
    pyautogui.press('down') 
    pyautogui.hotkey("ctrl","enter")
    time.sleep(0.5)
    Boton = boton_existe(session,"wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                    ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
            
    if Boton:
        print("Botón Borrar presente")
        # Boton borrar texto
        botonBorrar=session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                    ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
        botonBorrar.press()
        # Punto en el texto
        puntoentexto = session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:"
                                    "SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/cntlTEXT_EDITOR_0201/shellcont/shell")
        puntoentexto.text = "."
        pyautogui.hotkey("shift","TAB")
        time.sleep(0.5)
        pyautogui.press('down') 
        #session.sendVKey(2)
    else:
        print("Botón Borrar NO está en esta vista")


      # Navegar hasta el boton flecha abajo para siguiente posicion
    
    for i in range(2):
    pyautogui.hotkey("TAB")
    time.sleep(0.5)
    pyautogui.press("enter")


Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
